<a href="https://colab.research.google.com/github/hselino/complaint_analysis/blob/main/flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask groq flask-ngrok

In [ ]:
import torch
import threading
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from groq import Groq
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === Load models ===
summ_path = "/content/drive/MyDrive/prj/fine_tuned_model/summarizer"
bert1_path = "/content/drive/MyDrive/prj/fine_tuned_model/berturk-seg"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

summarizer_model = AutoModelForSeq2SeqLM.from_pretrained(summ_path).to(device)
summarizer_tokenizer = AutoTokenizer.from_pretrained(summ_path)

classifier_model = AutoModelForSequenceClassification.from_pretrained(bert1_path).to(device)
classifier_tokenizer = AutoTokenizer.from_pretrained(bert1_path)



In [ ]:
# === Groq client (replace with your actual API key) ===
groq_client = Groq(api_key="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")

df = pd.read_csv('/content/drive/MyDrive/prj/segmented_class.csv')

label_mapping = df.set_index('Unnamed: 0')['category'].to_dict()

# === Flask app ===
app = Flask(__name__)

In [ ]:
!pip install flask transformers groq
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip


--2025-05-06 07:57:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 75.2.60.68, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... ^C
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from sklearn.preprocessing import LabelEncoder
def summarize(text):
    inputs = summarizer_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    summary_ids = summarizer_model.generate(**inputs, max_length=128, num_beams=4)
    return summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def classify(text, label_encoder):
    inputs = classifier_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        logits = classifier_model(**inputs).logits
    predicted = torch.argmax(logits, dim=1).cpu().numpy()[0]
    return label_encoder.inverse_transform([predicted])[0]


label_encoder = LabelEncoder()

In [ ]:
@app.route('/generate_response', methods=['POST'])
def generate_response():
    data = request.get_json()
    complaint = data.get('complaint')

    if not complaint:
        return jsonify({'error': 'Missing complaint text'}), 400

    try:
        # === Classification ===
        classifier_inputs = classifier_tokenizer(complaint, return_tensors="pt").to(device)
        classifier_outputs = classifier_model(**classifier_inputs)
        predicted_class_id = classifier_outputs.logits.argmax().item()
        label = label_mapping.get(predicted_class_id, f"LABEL_{predicted_class_id}")

        # === Summarization ===
        summarizer_inputs = summarizer_tokenizer([complaint], return_tensors="pt").to(device)
        summary_ids = summarizer_model.generate(**summarizer_inputs, max_length=100, min_length=10)
        summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # === Groq LLaMA call ===
        prompt = f"""
        Kategori: {label}
        Özet: {summary}
        Şikayet: {complaint}

    Kullanıcıya Türkçe, kibar ve çözüm odaklı bir şekilde yanıt ver.

    """

        response = groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        generated_text = response.choices[0].message.content

        return jsonify({
            'label': label,
            'summary': summary,
            'response': generated_text
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# === Start Flask server in background ===
def start_flask():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=start_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 1s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [ ]:
!lt --port 5000


your url is: https://cold-regions-eat.loca.lt


INFO:werkzeug:127.0.0.1 - - [06/May/2025 08:42:45] "POST /generate_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 08:44:24] "POST /generate_response HTTP/1.1" 200 -


^C


In [ ]:
!./ngrok http 5000

Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018

